In [1]:
!python3 --version

Python 3.10.6


In [2]:
!python3 -m pip install --upgrade pip

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip3 install -U openai

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import json
import os
import sys

import openai

In [5]:
# OpenAIのapiキー
openai.api_key = 'OPENAI_KEY'
os.environ["OPENAI_API_KEY"] = openai.api_key

In [6]:
# 関数の実装
def get_belonging_prefecture(cities):
    """市町村名の羅列を受け取って、それぞれと都道府県の対応情報を持ったdictを返す。
        ただし特に定義のないものはモデルが学習した知識をそのまま使えるように「変更なし」を格納する"""
    def get_prefecture(city):
        return {"町田市": "神奈川県"}.get(city, "変更なし")

    prefecture_answer = [
        {"市区町村": city, "都道府県": get_prefecture(city)} for city in cities.split(",")
    ]
    return json.dumps(prefecture_answer)

In [7]:
# 使うことができる関数を羅列する
functions = [
    # 質問に対してこの関数を使うかどうか、
    # また使う時の引数は何にするかを判断するための情報を与える
    {
        "name": "get_belonging_prefecture",
        "description": "所属都道府県の変更情報を得る",
        "parameters": {
            "type": "object",
            "properties": {
                # cities引数の情報
                "cities": {
                    "type": "string",
                    "description": "市区町村名入力。半角カンマ区切りで複数要素を入力可能。各要素は「xx市」「xx区」「xx町」「xx村」のいずれか。例: 世田谷区,大阪市,府中町,山中湖村",
                },
            },
            "required": ["cities"],
        },
    }
]

In [13]:
def q_Chat(question):
    response = openai.ChatCompletion.create(
        # model="gpt-4-0613",
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": question},
        ],
        temperature=0.8,
        functions=functions,
        function_call="auto",
    )
    
    # print(json.dumps(response), file=sys.stderr)
    # print(json.dumps(response))
    print(response)
    
    message = response["choices"][0]["message"]
    
    if message.get("function_call"):
        # 関数を使用すると判断された場合

        # 使うと判断された関数名
        function_name = message["function_call"]["name"]
        # その時の引数dict
        arguments = json.loads(message["function_call"]["arguments"])

        # 2段階目の処理
        # 関数の実行
        function_response = get_belonging_prefecture(
            cities=arguments.get("cities"),
        )
        # print(function_response, file=sys.stderr)
        print(arguments)

        # 3段階目の処理
        # 関数実行結果を使ってもう一度質問
        second_response = openai.ChatCompletion.create(
            # model="gpt-4-0613",
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": question},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
            temperature=0.8,
        )

        # print(json.dumps(second_response), file=sys.stderr)
        # print(json.dumps(second_response))
        # print(second_response)
        print(second_response.choices[0]["message"]["content"])
        # print(second_response.choices[0]["message"]["content"].strip())
    else:
        # 関数を使用しない場合
        print(message["content"])
        # print(message["content"].strip())
        

In [14]:
question = "室町時代の赤松氏について、特に禁門の変の時期のことについて教えてください"

In [15]:
q_Chat(question)

{"id": "chatcmpl-7gh1CAbC2kPEt1W1xmF28X4FZNKPj", "object": "chat.completion", "created": 1690409494, "model": "gpt-3.5-turbo-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": "\u7981\u9580\u306e\u5909\u306f\u3001\u5ba4\u753a\u6642\u4ee3\u306e\u5fdc\u4ec1\u306e\u4e71\u3068\u3044\u3046\u5185\u4e71\u306e\u4e2d\u3067\u8d77\u3053\u3063\u305f\u4e8b\u4ef6\u3067\u3059\u3002\u8d64\u677e\u6c0f\u306f\u3001\u5ba4\u753a\u5e55\u5e9c\u306e\u7ba1\u9818\u3067\u3042\u3063\u305f\u7d30\u5ddd\u52dd\u5143\u306e\u5bb6\u65cf\u3067\u3042\u308a\u3001\u5fdc\u4ec1\u306e\u4e71\u306e\u4e2d\u3067\u4e3b\u5c0e\u7684\u306a\u5f79\u5272\u3092\u679c\u305f\u3057\u307e\u3057\u305f\u3002\n\n\u7981\u9580\u306e\u5909\u306f\u30011467\u5e74\u306b\u4eac\u90fd\u306e\u6771\u5bfa\u306e\u9580\u3067\u3042\u308b\u7981\u9580\u304c\u713c\u304b\u308c\u305f\u3053\u3068\u304b\u3089\u59cb\u307e\u308a\u307e\u3059\u3002\u3053\u306e\u4e8b\u4ef6\u306f\u3001\u7d30\u5ddd\u52dd\u5143\u3068\u5c71\u540d\u6301\u8c4a\u306e\u5b

In [14]:
question = "横浜市、町田市、相模原市、大磯町、これらの共通点は？"

In [15]:
q_Chat(question)

{"id": "chatcmpl-7gh2DkIkoCON4hM88xb79UU15RRv2", "object": "chat.completion", "created": 1690409557, "model": "gpt-3.5-turbo-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": null, "function_call": {"name": "get_belonging_prefecture", "arguments": "{\n  \"cities\": \"\u6a2a\u6d5c\u5e02,\u753a\u7530\u5e02,\u76f8\u6a21\u539f\u5e02,\u5927\u78ef\u753a\"\n}"}}, "finish_reason": "function_call"}], "usage": {"prompt_tokens": 186, "completion_tokens": 42, "total_tokens": 228}}
{
  "id": "chatcmpl-7gh2DkIkoCON4hM88xb79UU15RRv2",
  "object": "chat.completion",
  "created": 1690409557,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_belonging_prefecture",
          "arguments": "{\n  \"cities\": \"\u6a2a\u6d5c\u5e02,\u753a\u7530\u5e02,\u76f8\u6a21\u539f\u5e02,\u5927\u78ef\u753a\"\n}"
        }
      },
      "finish_reason": "